# LAB3 - Batch AI with Keras and CNTK backend


## Introduction

This recipe shows how to run Keras using Batch AI on DSVM. DSVM supports tensorflow, cntk and theano backends for running Keras. Currently only tensorflow and cntk backends supports running on GPU.

## Details

- DSVM has Keras framework preinstalled;
- Standard keras sample script mnist_cnn.py is used;
- The script downloads the standard MNIST Database on its own;
- Standard output of the job will be stored on Azure File Share.


## Instructions

### Install Dependencies and Create Configuration file.
```
> ssh sshuser@YOUR.VM.IP.ADDRESS
> az login
> az group create --name batchai_rg  --location eastus
> az storage account create --location eastus --name batchaipablo --resource-group batchai_rg --sku Standard_LRS
> az storage account keys list --account-name batchaipablo --resource-group batchai_rg -o table
> az ad sp create-for-rbac --name MyAppSvcPppl --password Passw0rd
> az storage account keys list --account-name batchaipablo --resource-group batchai_rg
```

### Read Configuration and Create Batch AI client

In [1]:
from __future__ import print_function

from datetime import datetime
import os
import sys
import zipfile

from azure.storage.file import FileService
import azure.mgmt.batchai.models as models

# utilities.py contains helper functions
import utilities

# Resource Group
location = 'eastus'
resource_group = 'batchai_rg'

# credentials used for authentication
client_id = 'ec0640c7-61fa-4662-bce4-8a3e931939ac'
secret = '/ScxxSeAlys0E94c2zBSVNr8yjGPjRcOn9hGQiG6z/4='
secret = 'Passw0rd'
token_uri = 'https://login.microsoftonline.com/72f988bf-86f1-41af-91ab-2d7cd011db47/oauth2/token'
subscription_id = 'b1395605-1fe9-4af4-b3ff-82a4725a3791'

# credentials used for storage
storage_account_name = 'batchaipablo'
storage_account_key = 'y59heteYEbw5nTLBB/b7rj3jUphvs2Iwslg4AsXFSb4G7ZLgJUep4AuccSmST7I3E8Zw4BaUloebK+VyKmGpog=='

# specify the credentials used to remote login your GPU node
admin_user_name = 'sshuser'
admin_user_password = 'Passw0rd.1!!'

In [2]:
from azure.common.credentials import ServicePrincipalCredentials
import azure.mgmt.batchai as batchai
import azure.mgmt.batchai.models as models

creds = ServicePrincipalCredentials(client_id=client_id, secret=secret, token_uri=token_uri)

client = batchai.BatchAIManagementClient(credentials=creds,subscription_id=subscription_id)

### Create File Share

For this example we will create a new File Share with name `batchaisample` under your storage account.

**Note** You don't need to create new file share for every cluster. We are doing this in this sample to simplify resource management for you.

In [4]:
azure_file_share_name = 'batchailab3'
service = FileService(storage_account_name, storage_account_key)
service.create_share(azure_file_share_name, fail_on_exist=False)
print('Done')

Done


### Configure Compute Cluster

- For this example we will use a gpu cluster of `STANDARD_NC6` nodes. Number of nodes in the cluster is configured with `nodes_count` variable;
- We will mount file share at folder with name `external`. Full path of this folder on a computer node will be `$AZ_BATCHAI_MOUNT_ROOT/external`;
- We will call the cluster `nc6`;

So, the cluster will have the following parameters:

In [5]:
azure_file_share = 'external'
nodes_count = 2
cluster_name = 'nc6'
vmsize = "Standard_NC6"

volumes = models.MountVolumes(
    azure_file_shares=[
        models.AzureFileShareReference(
            account_name=storage_account_name,
            credentials=models.AzureStorageCredentialsInfo(
                account_key=storage_account_key),
            azure_file_url = 'https://{0}.file.core.windows.net/{1}'.format(
                storage_account_name, azure_file_share_name),
            relative_mount_path=azure_file_share)
    ]
)

parameters = models.ClusterCreateParameters(
    location=location,
    vm_size=vmsize,
    virtual_machine_configuration=models.VirtualMachineConfiguration(
        image_reference=models.ImageReference(
            publisher="microsoft-ads",
            offer="linux-data-science-vm-ubuntu",
            sku="linuxdsvmubuntu",
            version="latest")),    
    user_account_settings=models.UserAccountSettings(
        admin_user_name=admin_user_name,
        admin_user_password=admin_user_password),
    scale_settings=models.ScaleSettings(
        manual=models.ManualScaleSettings(target_node_count=nodes_count)
    ),
    node_setup=models.NodeSetup(
        mount_volumes=volumes,
    )
)

### Create Compute Cluster

In [6]:
cluster = client.clusters.create(resource_group, cluster_name, parameters).result()

### Monitor Cluster Creation

Monitor the just created cluster. utilities.py contains a helper function to print out detail status of the cluster.

In [7]:
cluster = client.clusters.get(resource_group, cluster_name)
utilities.print_cluster_status(cluster)

Cluster state: AllocationState.resizing Target: 2; Allocated: 0; Idle: 0; Unusable: 0; Running: 0; Preparing: 0; Leaving: 0


### Deploy Sample Script and Configure the Input Directories

- Download original sample script:

In [8]:
sample_script_url = 'https://raw.githubusercontent.com/fchollet/keras/master/examples/mnist_cnn.py'
utilities.download_file(sample_script_url, 'mnist_cnn.py')

- For each job we will create a folder containing a copy of train_mnist.py. This allows each job to have it's own copy of the sample script (in case you would like to change it).

In [9]:
keras_sample_dir = "KerasSamples"
service = FileService(storage_account_name, storage_account_key)
service.create_directory(
    azure_file_share_name, keras_sample_dir, fail_on_exist=False)
service.create_file_from_path(
    azure_file_share_name, keras_sample_dir, 'mnist_cnn.py', 'mnist_cnn.py')
print('Done')

Done


-The job needs to know where to find train_mnist.py script (the script will download MNIST dataset on its own). So, we will configure an input directory for the script:

In [10]:
input_directories = [
    models.InputDirectory(
        id='SCRIPT',
        path='$AZ_BATCHAI_MOUNT_ROOT/{0}/{1}'.format(azure_file_share, keras_sample_dir))
]
output_directories = [
    models.OutputDirectory(
        id='MODEL',
        path_prefix='$AZ_BATCHAI_MOUNT_ROOT/{0}'.format(azure_file_share),
        path_suffix="Models")]

The job will be able to reference those directories using ```$AZ_BATCHAI_INPUT_SCRIPT``` and ```$AZ_BATCHAI_INPUT_DATASET``` environment variables.

### Configure Output Directories
We will store standard and error output of the job in File Share:

In [11]:
std_output_path_prefix = "$AZ_BATCHAI_MOUNT_ROOT/{0}".format(azure_file_share)

### Configure Job
- Will use configured previously input and output directories;
- Will run standard mnist_cnn.py from SCRIPT input directory using custom framework;
- Keras will use theano backend; DSVM supports cntk, tensorflow and theano backends for keras, just change KERAS_BACKEND to "tensorflow" or "cntk" to use corresponding backend. Note, theano backend will run on CPU. 
- Will output standard output and error streams to file share.

In [13]:
job_name = datetime.utcnow().strftime("keras_%m_%d_%Y_%H%M%S")
parameters = models.job_create_parameters.JobCreateParameters(
     location=location,
     cluster=models.ResourceId(cluster.id),
     node_count=nodes_count,
     input_directories=input_directories,
     output_directories=output_directories,
     std_out_err_path_prefix=std_output_path_prefix,
     custom_toolkit_settings = models.CustomToolkitSettings(
         command_line='KERAS_BACKEND=cntk python $AZ_BATCHAI_INPUT_SCRIPT/mnist_cnn.py'))

### Create a training Job and wait for Job completion


In [14]:
job = client.jobs.create(resource_group, job_name, parameters).result()
print('Created Job: {}'.format(job_name))

Created Job: keras_02_13_2018_225916


### Wait for Job to Finish
The job will start running when the cluster will have enought idle nodes. The following code waits for job to start running printing the cluster state. During job run, the code prints current content of stdout.

**Note** Execution may take several minutes to complete.

In [15]:
utilities.wait_for_job_completion(client, resource_group, job_name, cluster_name, 'stdOuterr', 'stdout.txt')

Cluster state: AllocationState.steady Target: 2; Allocated: 2; Idle: 0; Unusable: 0; Running: 2; Preparing: 0; Leaving: 0
Job state: running ExitCode: None
Waiting for job output to become available...

11493376/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/12

31232/60000 [==============>...............] - ETA: 12s - loss: 0.4910 - acc: 0.8492


55808/60000 [==========================>...] - ETA: 1s - loss: 0.3473 - acc: 0.8938


60000/60000 [==============================] - 23s 378us/step - loss: 0.3329 - acc: 0.8985 - val_loss: 0.0765 - val_acc: 0.9755
Epoch 2/12

22144/60000 [==========>...................] - ETA: 10s - loss: 0.1213 - acc: 0.9643


48768/60000 [=======================>......] - ETA: 3s - loss: 0.1151 - acc: 0.9663


60000/60000 [==============================] - 17s 289us/step - loss: 0.1122 - acc: 0.9670 - val_loss: 0.0519 - val_acc: 0.9832
Epoch 3/12

16128/60000 [=======>......................] - ETA: 12s - loss: 0.0856 - acc: 0.9748


...

60000/60000 [==============================] - 20s 329us/step - loss: 0.0398 - acc: 0.9881 - val_loss: 0.0306 - val_acc: 0.9900
Epoch 11/12

17408/60000 [=======>......................] - ETA: 11s - loss: 0.0338 - acc: 0.9898


39936/60000 [==================>...........] - ETA: 6s - loss: 0.0362 - acc: 0.9893


60000/60000 [==============================] - 19s 324us/step - loss: 0.0366 - acc: 0.9892 - val_loss: 0.0286 - val_acc: 0.9899
Epoch 12/12

 6400/60000 [==>...........................] - ETA: 16s - loss: 0.0293 - acc: 0.9900


29568/60000 [=============>................] - ETA: 9s - loss: 0.0366 - acc: 0.9885


60000/60000 [==============================] - 20s 342us/step - loss: 0.0356 - acc: 0.9893 - val_loss: 0.0274 - val_acc: 0.9904
Test loss: 0.0273927967413
Test accuracy: 0.9904
Job state: succeeded ExitCode: 0


### Download stdout.txt and stderr.txt files for the Job

In [16]:
files = client.jobs.list_output_files(resource_group, job_name, models.JobsListOutputFilesOptions("stdOuterr")) 
for f in list(files):
    utilities.download_file(f.download_url, f.name)
print("All files downloaded")

All files downloaded


### Delete the Job

In [17]:
_ = client.jobs.delete(resource_group, job_name)

### Delete the Cluster
When you are finished with the sample and don't want to submit any more jobs you can delete the cluster using the following code.

In [18]:
_ = client.clusters.delete(resource_group, cluster_name)

### Delete File Share
When you are finished with the sample and don't want to submit any more jobs you can delete the file share completely with all files using the following code.

In [19]:
service = FileService(storage_account_name, storage_account_key)
service.delete_share(azure_file_share_name)

True